In [1]:
import subprocess
import findspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode
from pyspark.sql.functions import split
from pyspark.sql.functions import from_unixtime
from pyspark.sql.functions import from_json
from pyspark.sql.types import *

In [2]:
findspark.init(spark_home='/opt/spark')

In [3]:
topics = subprocess.check_output("/opt/kafka/bin/kafka-topics.sh --list --bootstrap-server hadoop-namenode:9092", shell=True)
topics = topics.split()
topics = [ topic.decode("UTF-8") for topic in topics ]
topics.pop()
topics

['XBEESmartParkingModel']

In [4]:
schema = StructType([StructField("parking_name", StringType())])

In [5]:
# Spark session & context
spark = (SparkSession
         .builder
         .master('local')
         .appName('Parkings')
         .config('spark.jars', 'file:///opt/smart-parking/Python/GetData/spark-sql-kafka-0-10_2.12-3.2.1.jar,file:///opt/smart-parking/Python/GetData/kafka-clients-3.1.0.jar')
         .config('spark.executor.extraClassPath','file:///opt/smart-parking/Python/GetData/spark-sql-kafka-0-10_2.12-3.2.1.jar:file:///opt/smart-parking/Python/GetData/kafka-clients-3.1.0.jar')
         .config('spark.executor.extraLibrary','file:///opt/smart-parking/Python/GetData/spark-sql-kafka-0-10_2.12-3.2.1.jar:file:///opt/smart-parking/Python/GetData/kafka-clients-3.1.0.jar')
         .config('spark.driver.extraClassPath', 'file:///opt/smart-parking/Python/GetData/spark-sql-kafka-0-10_2.12-3.2.1.jar:file:///opt/smart-parking/Python/GetData/kafka-clients-3.1.0.jar')
         .getOrCreate())

spark.sparkContext.setLogLevel("ERROR")
sc = spark.sparkContext

22/05/16 13:28:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [6]:
# Subscribe to 1 topic
df = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "hadoop-namenode:9092") \
  .option("subscribe", "XBEESmartParkingModel") \
  .load()
df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")

DataFrame[key: string, value: string]

In [7]:
decode_value = df[['value']]
decode_value = decode_value.withColumn("value", decode_value.value.cast("String"))

In [8]:
decode_value.select(from_json(decode_value.value, schema).alias("json"))

DataFrame[json: struct<parking_name:string>]

In [9]:
# Inicia la consulta e imprime el resultado
query = decode_value \
    .writeStream \
    .format("memory") \
    .queryName("Parkings") \
    .start()

In [10]:
from IPython.display import display, clear_output
from time import sleep

while True:
    clear_output(wait=True)
    display(query.status)
    display(spark.sql('SELECT * FROM Parkings ').show(truncate=False))
    sleep(1)

{'message': 'Getting offsets from KafkaV2[Subscribe[XBEESmartParkingModel]]',
 'isDataAvailable': False,
 'isTriggerActive': True}

+-----+
|value|
+-----+
+-----+



None

KeyboardInterrupt: 